In [1]:
import os

from groq import Groq

from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chains import SequentialChain
import json

from datetime import datetime, timedelta
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import pandas as pd
import traceback
from langchain.document_loaders import PyPDFLoader
import pypdf
from PyPDF2 import PdfReader
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
KEY = os.getenv('GROQ_API_KEY')

In [4]:
# model="llama-3.1-70b-versatile"
groq_api_key = KEY  # Replace with your Groq API key  model="llama-3.1-8b-instant"  model="llama-3.1-8b-instant"  llama3-70b-8192
model = "llama3-70b-8192"
groq_chat = ChatGroq(groq_api_key=groq_api_key, model_name=model,temperature=0.5)

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
"""
number=5
subject="Data Science And Machine Learning"
tone="Medium"
"""

'\nnumber=5\nsubject="Data Science And Machine Learning"\ntone="Medium"\n'

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [9]:
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n')

In [10]:
quiz_chain=LLMChain(llm=groq_chat, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\somva\Documents\Work\Foundational_Generative_AI\MCQ_GENERATOR_Llama_3.1\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [14]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F17BAC0DD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F1794E04D0>, model_name='llama3-70b-8192', temperature=0.5, groq_api_key=SecretStr('**********')), output_key='quiz')

In [15]:
TEMPLATE_1="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE_1)

In [17]:
review_chain=LLMChain(llm=groq_chat, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
review_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.\nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F17BAC0DD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F1794E04D0>, model_name='llama3-70b-8192', temperature=0.5, groq_api_key=SecretStr('**********')), output_key='review')

In [19]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [20]:
path = 'data.txt'

In [21]:
with open(path, 'r') as file:
    TEXT = file.read()

In [22]:
print(TEXT)

Machine learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. Instead of being programmed to carry out specific tasks, machine learning systems learn from data, identifying patterns and making decisions based on that data. Here's a detailed breakdown of what machine learning entails:

1. Types of Machine Learning
a. Supervised Learning:
In supervised learning, the algorithm is trained on a labeled dataset, meaning that each training example is paired with an output label. The goal is to learn a mapping from inputs to outputs that can be used to predict labels for new, unseen data. Supervised learning tasks are further divided into:

Regression: Predicting continuous values. Examples include predicting house prices or stock market trends.
Classification: Predicting discrete labels. Examples include spam detection (spam or not spam) and image reco

In [23]:
number=5
subject="Data Science And Machine Learning"
tone="Medium"

In [24]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": number,
            "subject":subject,
            "tone": tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\somva\Documents\Work\Foundational_Generative_AI\MCQ_GENERATOR_Llama_3.1\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. Instead of being programmed to carry out specific tasks, machine learning systems learn from data, identifying patterns and making decisions based on that data. Here's a detailed breakdown of what machine learning entails:

1. Types of Machine Learning
a. Supervised Learning:
In supervised learning, the algorithm is trained on a labeled dataset, meaning that each training example is paired with an output label. The goal is to learn a mapping from inputs to outputs that can be used to predict labels for new, unseen data. Supervised learning tasks are further divided into:

Regression: Predicting continuous values. Examples include predicting house prices or stock market trends.
C

In [25]:
response

{'text': "Machine learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. Instead of being programmed to carry out specific tasks, machine learning systems learn from data, identifying patterns and making decisions based on that data. Here's a detailed breakdown of what machine learning entails:\n\n1. Types of Machine Learning\na. Supervised Learning:\nIn supervised learning, the algorithm is trained on a labeled dataset, meaning that each training example is paired with an output label. The goal is to learn a mapping from inputs to outputs that can be used to predict labels for new, unseen data. Supervised learning tasks are further divided into:\n\nRegression: Predicting continuous values. Examples include predicting house prices or stock market trends.\nClassification: Predicting discrete labels. Examples include spam detection (spam or not spa

In [26]:
quiz = response.get("quiz")

In [27]:
quiz

'Here are 5 multiple-choice questions based on the provided text:\n\n{"1": {"mcq": "What is the primary goal of supervised learning in machine learning?", "options": {"a": "To identify patterns in unlabeled data", "b": "To learn a mapping from inputs to outputs", "c": "To make decisions by performing actions in an environment", "d": "To group data points into clusters"}, "correct": "b"} ,\n\n"2": {"mcq": "Which type of machine learning involves an agent learning to make decisions by performing actions in an environment to maximize cumulative reward?", "options": {"a": "Supervised learning", "b": "Unsupervised learning", "c": "Semi-supervised learning", "d": "Reinforcement learning"}, "correct": "d"} ,\n\n"3": {"mcq": "What is the purpose of feature engineering in machine learning?", "options": {"a": "To reduce the dimensionality of the data", "b": "To create new features or transform existing features to improve model performance", "c": "To evaluate the performance of a model", "d": "T

In [28]:
print(quiz)

Here are 5 multiple-choice questions based on the provided text:

{"1": {"mcq": "What is the primary goal of supervised learning in machine learning?", "options": {"a": "To identify patterns in unlabeled data", "b": "To learn a mapping from inputs to outputs", "c": "To make decisions by performing actions in an environment", "d": "To group data points into clusters"}, "correct": "b"} ,

"2": {"mcq": "Which type of machine learning involves an agent learning to make decisions by performing actions in an environment to maximize cumulative reward?", "options": {"a": "Supervised learning", "b": "Unsupervised learning", "c": "Semi-supervised learning", "d": "Reinforcement learning"}, "correct": "d"} ,

"3": {"mcq": "What is the purpose of feature engineering in machine learning?", "options": {"a": "To reduce the dimensionality of the data", "b": "To create new features or transform existing features to improve model performance", "c": "To evaluate the performance of a model", "d": "To selec

In [29]:
type(quiz)

str

In [39]:
def extract_mcqs(input_string):
    # Remove the introductory text and get the JSON part
    json_string = input_string.split('\n\n', 1)[1]
    
    # Initialize an empty dictionary to store the MCQs
    mcqs = {}
    
    # Split the string into individual MCQ blocks
    mcq_blocks = json_string.split('} ,\n\n')
    
    for block in mcq_blocks:
        # Extract the question number
        question_num = block.split(':')[0].strip('"{')
        
        # Extract the MCQ question
        mcq_start = block.find('"mcq": "') + 8
        mcq_end = block.find('", "options"')
        mcq_question = block[mcq_start:mcq_end]
        
        # Extract the options
        options_start = block.find('"options": {') + 11
        options_end = block.rfind('}, "correct"')
        options_string = block[options_start:options_end]
        
        options = {}
        option_pairs = options_string.split('", "')
        for pair in option_pairs:
            if '": "' in pair:
                key, value = pair.split('": "', 1)
                key = key.strip('"{}').strip()
                value = value.strip('"')
                options[key] = value
        
        # Extract the correct answer
        correct_start = block.rfind('"correct": "') + 12
        correct_end = block.rfind('"')
        correct_answer = block[correct_start:correct_end]
        
        # Store the extracted information in the dictionary
        mcqs[question_num] = {
            'question': mcq_question,
            'options': options,
            'correct_answer': correct_answer
        }
    
    return mcqs

In [40]:

extracted_mcqs = extract_mcqs(quiz)


In [41]:

# Print the extracted MCQs
for num, mcq in extracted_mcqs.items():
    print(f"Question {num}:")
    print(f"  Question: {mcq['question']}")
    print("  Options:")
    for key, value in mcq['options'].items():
        print(f"    {key}: {value}")
    print(f"  Correct Answer: {mcq['correct_answer']}")
    print()



Question 1:
  Question: What is the primary goal of supervised learning in machine learning?
  Options:
    a: To identify patterns in unlabeled data
    b: To learn a mapping from inputs to outputs
    c: To make decisions by performing actions in an environment
    d: To group data points into clusters
  Correct Answer: b

Question 2:
  Question: Which type of machine learning involves an agent learning to make decisions by performing actions in an environment to maximize cumulative reward?
  Options:
    a: Supervised learning
    b: Unsupervised learning
    c: Semi-supervised learning
    d: Reinforcement learning
  Correct Answer: d

Question 3:
  Question: What is the purpose of feature engineering in machine learning?
  Options:
    a: To reduce the dimensionality of the data
    b: To create new features or transform existing features to improve model performance
    c: To evaluate the performance of a model
    d: To select the most important features for the model
  Correct 

In [45]:
extracted_mcqs

{'1': {'question': 'What is the primary goal of supervised learning in machine learning?',
  'options': {'a': 'To identify patterns in unlabeled data',
   'b': 'To learn a mapping from inputs to outputs',
   'c': 'To make decisions by performing actions in an environment',
   'd': 'To group data points into clusters'},
  'correct_answer': 'b'},
 '2': {'question': 'Which type of machine learning involves an agent learning to make decisions by performing actions in an environment to maximize cumulative reward?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Semi-supervised learning',
   'd': 'Reinforcement learning'},
  'correct_answer': 'd'},
 '3': {'question': 'What is the purpose of feature engineering in machine learning?',
  'options': {'a': 'To reduce the dimensionality of the data',
   'b': 'To create new features or transform existing features to improve model performance',
   'c': 'To evaluate the performance of a model',
   'd': 'To select 

In [46]:
# Convert to DataFrame

def create_mcq_dataframe(mcqs):
    flattened_data = []
    for num, mcq in mcqs.items():
        row = {
            'question_number': num,
            'question': mcq['question'],
            'correct_answer': mcq['correct_answer']
        }
        for option, text in mcq['options'].items():
            row[f'option_{option}'] = text
        flattened_data.append(row)
    
    return pd.DataFrame(flattened_data)



In [47]:
# Create and print the DataFrame
df = create_mcq_dataframe(extracted_mcqs)


In [48]:
df

,question_number,question,correct_answer,option_a,option_b,option_c,option_d
0,1,What is the primary goal of supervised learnin...,b,To identify patterns in unlabeled data,To learn a mapping from inputs to outputs,To make decisions by performing actions in an ...,To group data points into clusters
1,2,Which type of machine learning involves an age...,d,Supervised learning,Unsupervised learning,Semi-supervised learning,Reinforcement learning
2,3,What is the purpose of feature engineering in ...,b,To reduce the dimensionality of the data,To create new features or transform existing f...,To evaluate the performance of a model,To select the most important features for the ...
3,4,What is the term for a model that learns the n...,a,Overfitting,Underfitting,Supervised learning,Unsupervised learning
4,5,Which of the following is an application of ma...,c,Predicting stock market trends,Customer segmentation,Predicting disease outbreaks,Route optimization


In [49]:
# Print options for the first question
print("\nOptions for the first question:")
print(df.loc[0, ['option_a', 'option_b', 'option_c', 'option_d']])


Options for the first question:
option_a               To identify patterns in unlabeled data
option_b            To learn a mapping from inputs to outputs
option_c    To make decisions by performing actions in an ...
option_d                   To group data points into clusters
Name: 0, dtype: object
